This is a sample for celestial navigation for a stationary observer<br> 
© August Linnman, 2025, email: august@linnman.net<br>
MIT License (see [LICENSE file](LICENSE))

Jupyter notebook for ordinary three-fix celestial navigation (stationary).

Run it once to create the input form. Used data will be saved for later sessions. 

In [5]:
# Importing Python libraries
from time import time
from starfix import Sight, SightCollection, get_representation, get_google_map_string,\
                    IntersectError, LatLonGeodetic

In [ ]:
import notebook_helper
from ipywidgets import VBox

notebook_helper.initialize ("notebook.1.json",
                {"ObjectName1" : "Capella",
               "Altitude1" : "33:9:34",
               "Time1" : "2024-09-17 23:36:13+00:00",
               "LimbCorrection1" : "0",
               "IndexError1" : "0",
               "ArtificialHorizon1" : "False",
               "ObserverHeight1" : "0",
               "Temperature1" : "10",
               "TemperatureGradient1" : "-0.01",
               "Pressure1" : "101",
               
               "ObjectName2" : "Moon",
               "Altitude2" : "48:22:5.2",
               "Time2" : "2024-09-17 23:41:13+00:00",
               "LimbCorrection2" : "0",
               "IndexError2" : "0",
               "ArtificialHorizon2" : "False",
               "ObserverHeight2" : "0",
               "Temperature2" : "10",
               "TemperatureGradient2" : "-0.01",
               "Pressure2" : "101",                                             
               
               "ObjectName3" : "Vega",
               "Altitude3" : "25:39:4",
               "Time3" : "2024-09-17 23:46:13+00:00",
               "LimbCorrection3" : "0",
               "IndexError3" : "0",
               "ArtificialHorizon3" : "False",
               "ObserverHeight3" : "0",               
               "Temperature3" : "10",
               "TemperatureGradient3" : "-0.01",                
               "Pressure3" : "101",               

               "DrpLat" : "36",
               "DrpLon" : "10",
               
               "Use1" : "True",
               "Use2" : "True",
               "Use3" : "True"})


w = notebook_helper.render_widget()
VBox(w)

In [ ]:
# SIGHTS

from notebook_helper import str2bool as str2bool
numDict = notebook_helper.NUM_DICT
typeArray = notebook_helper.TYPE_ARRAY
assert isinstance (typeArray, list)

def get_starfixes (drp_pos : LatLonGeodetic) -> SightCollection :
    ''' Returns a list of used star fixes (SightCollection) '''

    Sight.set_estimated_position (drp_pos)
    retval = []
    for i in range (3):
        if str2bool(numDict["Use"+str(i+1)]):
            retval.append (Sight ( object_name          = numDict         [typeArray[0][0]+str(i+1)],
                                   measured_alt         = numDict         [typeArray[1][0]+str(i+1)],
                                   set_time             = numDict         [typeArray[2][0]+str(i+1)],
                                   index_error_minutes  = float(numDict   [typeArray[3][0]+str(i+1)]),
                                   limb_correction      = int(numDict     [typeArray[4][0]+str(i+1)]),
                                   artificial_horizon   = str2bool(numDict[typeArray[5][0]+str(i+1)]),
                                   observer_height      = float(numDict   [typeArray[6][0]+str(i+1)]),
                                   temperature          = float(numDict   [typeArray[7][0]+str(i+1)]),
                                   dt_dh                = float(numDict   [typeArray[8][0]+str(i+1)]),
                                   pressure             = float(numDict   [typeArray[9][0]+str(i+1)])
                                 ))

    return SightCollection (retval)


In [ ]:
# SIGHT REDUCTION.

the_pos = LatLonGeodetic (float(numDict["DrpLat"]), 
                          float(numDict["DrpLon"])) # Rough DRP position

intersections = None
collection = None
the_map = None
try:
    intersections, _, _, collection =\
            SightCollection.get_intersections_conv (return_geodetic=True,
                                                    estimated_position=the_pos,
                                                    get_starfixes=get_starfixes)
    assert intersections is not None
    assert collection is not None
    print (get_representation(intersections,1))
    assert isinstance (intersections, LatLonGeodetic)
    print ("Google Map Coordinate = " + get_google_map_string(intersections,4)) 

except IntersectError as ve:
    print ("Cannot perform a sight reduction. Bad sight data.\n" + str(ve))
    if ve.coll_object is not None:
        if isinstance (ve.coll_object, SightCollection):
            collection = ve.coll_object

if isinstance (intersections, tuple):
    intersections = None

if collection is not None:
    the_map = collection.render_folium (intersections)
the_map
